<table align="left">
  <tr>
    <td><img src="Museum.png" alt="ice cream truck in front of Louvre" width="120"/></td>
    <td align="left"><h1>Lesson 5: Building a Deep Research-like agent</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> :</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Set up and test search tools

In [1]:
from tavily import TavilyClient
from smolagents import tool, Tool
import os
from dotenv import load_dotenv

load_dotenv(override=True) # load variables from local .env file

@tool
def web_search(query: str) -> str:
    """Searches the web for your query.

    Args:
        query: Your query
    """
    tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    response = tavily_client.search(query)
    return str(response["results"])


class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, 40000)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

print(web_search("What is the melting temperature of vanilla ice cream in °C?"))

[{'title': 'What Is the Melting Point of Ice Cream? - Reference.com', 'url': 'https://www.reference.com/science-technology/melting-point-ice-cream-d438071a06dd9e19', 'content': 'What Is the Melting Point of Ice Cream? According to the University of California – Santa Barbra, ice cream will melt at about 31 degrees Fahrenheit or -3 degrees Celsius. The freezing point of water is about 32 degrees Fahrenheit, but the ice cream is affected by the salt content. Adding salt to a liquid lowers the freezing point of the substance. The reason why salt affects the freezing point of liquids is because of the way liquids freeze. The molecules in a liquid are constantly moving, but as the liquid becomes colder, the molecules slow down causing it to freeze. When salt molecules are added to the liquid, the two molecules combine so that more heat must be removed in order for the liquid to freeze.', 'score': 0.5820878, 'raw_content': None}, {'title': "What temperature does ice cream melt? - Chef's Reso

## Set up an agent

In [2]:
from PIL import Image

from smolagents import CodeAgent, OpenAIServerModel

from huggingface_hub import login

login(os.getenv('HF_API_KEY'))

code_model = "gpt-4.1-mini"

model = OpenAIServerModel(
    #"gpt-4o",
    code_model,  #updated model
    max_completion_tokens=8096,
)

<div style="font-size: 20px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> In the notebook, we reduced the museum count from 10 to 3 to reduce runtime and cost.  
  This will reduce the number of ice cream trucks you will need to buy to start your business!
</div>


In [3]:
#changed to 2024 to avoid decaying internet content
request_museums = """
Could you give me a sorted list of the top 3 museums in the world in 2024,
along with their visitor count (in millions) that year, and the approximate daily temperature
in July at their location ?"""

In [4]:
agent = CodeAgent(
    model=model,
    tools=[web_search, VisitWebpageTool()],
    max_steps=10
)
agent.logger.console.width=66
result = agent.run(request_museums)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_3_museums_2024 = web_search(query="top 3 most visited       
  museums in the world 2024 visitor count")                       
  print(top_3_museums_2024)                                       
 ────────────────────────────────────────────────────────────────

Execution logs:
[{'title': 'These were the most-visited museums around the world 
in 2024', 'url': 
'https://www.wanderlustmagazine.com/news/most-visited-museums-2024
/', 'content': 'These were the most-visited museums around the 
world in 2024 - Wanderlust [![Image 
10](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_CultureHeritage.svg)](https://www.wanderlustmagaz
ine.com/inspiration/culture-heritage/)[Culture & 
Heritage](https://www.wanderlustmagazine.com/inspiration/culture-h
eritage/) [![Image 
11](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SustainableTravel.svg)](https://www.wanderlustmag
azine.com/inspiration/sustainable-travel/)[Sustainable 
Travel](https://www.wanderlustmagazine.com/inspiration/sustainable
-travel/) [![Image 
12](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Rail.svg)](https://www.wanderlustmagazine.com/ins
piration/rail/)[Rail](https://www.wanderlustmagazine.com/inspirati
on/rail/) [![Image 
13](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SoloTravel.svg)](https://www.wanderlustmagazine.c
om/inspiration/solo-travel/)[Solo 
Travel](https://www.wanderlustmagazine.com/inspiration/solo-travel
/) [![Image 
14](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_OutdoorsWalking.svg)](https://www.wanderlustmagaz
ine.com/inspiration/outdoors-walking/)[Outdoors & 
Walking](https://www.wanderlustmagazine.com/inspiration/outdoors-w
alking/) [![Image 
15](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Sleeps.svg)](https://www.wanderlustmagazine.com/i
nspiration/sleeps/)[Sleeps](https://www.wanderlustmagazine.com/ins
piration/sleeps/) [![Image 
16](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_NatureWildlife.svg)](https://www.wanderlustmagazi
ne.com/inspiration/nature-wildlife/)[Nature & 
Wildlife](https://www.wanderlustmagazine.com/inspiration/nature-wi
ldlife/) [![Image 
17](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_FoodDrink.svg)](https://www.wanderlustmagazine.co
m/inspiration/food-drink/)[Food & 
Drink](https://www.wanderlustmagazine.com/inspiration/food-drink/)
[![Image 
18](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Trips.svg)](https://www.wanderlustmagazine.com/in
spiration/trips/)[Trips](https://www.wanderlustmagazine.com/inspir
ation/trips/) [![Image 
19](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_PromotedJourneys.svg)](https://www.wanderlustmaga
zine.com/inspiration/promoted-journeys/)[Promoted 
Journeys](https://www.wanderlustmagazine.com/inspiration/promoted-
journeys/) [![Image 
28](https://www.wanderlustmagazine.com/wp-content/uploads/2025/04/
WL238-Cover-1-151x209.jpg)](https://www.wanderlustmagazine.com/mag
azine/april-may-2025/)[April/May 
2025](https://www.wanderlustmagazine.com/magazine/april-may-2025/)
[View 
issue](https://www.wanderlustmagazine.com/magazine/april-may-2025/
) [![Image 
30](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_CultureHeritage.svg)](https://www.wanderlustmagaz
ine.com/inspiration/culture-heritage/)[Culture & 
Heritage](https://www.wanderlustmagazine.com/inspiration/culture-h
eritage/) [![Image 
31](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SustainableTravel.svg)](https://www.wanderlustmag
azine.com/inspiration/sustainable-travel/)[Sustainable 
Travel](https://www.wanderlustmagazine.com/inspiration/sustainable
-travel/) [![Image 
32](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_Rail.svg)](https://www.wanderlustmagazine.com/ins
piration/rail/)[Rail](https://www.wanderlustmagazine.com/inspirati
on/rail/) [![Image 
33](https://www.wanderlustmagazine.com/wp-content/uploads/2023/09/
WAND_Inspiration_SoloTravel.svg)](https://www.

[Step 1: Duration 2.70 seconds| Input tokens: 2,097 | Output 
tokens: 134]

...

[Step 8: Duration 7.64 seconds| Input tokens: 54,889 | Output 
tokens: 1,068]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  top_3_museums_info = [                                          
      {"museum": "Louvre Museum", "location": "Paris",            
  "visitor_count_million": 9, "avg_daily_temp_july_C": 24.6},     
      {"museum": "Vatican Museum", "location": "Vatican City",    
  "visitor_count_million": 6.8, "avg_daily_temp_july_C": 31},     
      {"museum": "British Museum", "location": "London",          
  "visitor_count_million": 6.5, "avg_daily_temp_july_C": 21},  #  
  Approx 21°C average July temp                                   
  ]                                                               
                                                                  
  # Sort by visitor count descending (already sorted)             
  final_answer(top_3_museums_info)                                
 ────────────────────────────────────────────────────────────────

Out - Final answer: [{'museum': 'Louvre Museum', 'location': 
'Paris', 'visitor_count_million': 9, 'avg_daily_temp_july_C': 
24.6}, {'museum': 'Vatican Museum', 'location': 'Vatican City', 
'visitor_count_million': 6.8, 'avg_daily_temp_july_C': 31}, 
{'museum': 'British Museum', 'location': 'London', 
'visitor_count_million': 6.5, 'avg_daily_temp_july_C': 21}]

[Step 9: Duration 0.05 seconds| Input tokens: 68,884 | Output 
tokens: 1,308]

In [5]:
#modified code to handle case where agent doesn't return a python list.
import pandas as pd

try:
    display(pd.DataFrame(result))
except Exception as e:
    print("Could not display as DataFrame:", e)
    print(result)

,museum,location,visitor_count_million,avg_daily_temp_july_C
0,Louvre Museum,Paris,9.0,24.6
1,Vatican Museum,Vatican City,6.8,31.0
2,British Museum,London,6.5,21.0


## Create a multi-agent researcher
### Start with web search agent

In [6]:
web_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,  # updated model
        max_completion_tokens=8096,
    ),
    tools=[web_search, VisitWebpageTool()],
    max_steps=10,
    name="web_agent",
    description="Runs web searches for you."
)
web_agent.logger.console.width=66

### Setup manager agent

In [7]:
from smolagents import Tool
from typing import Any
from smolagents.utils import make_image_url, encode_image_base64

def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel(
        "gpt-4o",
    )
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
        "Also, any run that invents numbers should fail."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=OpenAIServerModel(
        #"gpt-4o",
        code_model,
        max_tokens=8096,
    ),
    tools=[],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "plotly.express",  #added
        "plotly.express.colors",  #added
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)
manager_agent.logger.console.width=66

In [8]:
manager_agent.visualize()

CodeAgent | gpt-4.1-mini
├── ✅ Authorized imports: ['geopandas', 'plotly', 
│   'plotly.express', 'plotly.express.colors', 'shapely', 'json', 
│   'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description          ┃ Arguments            ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final     │ answer (`any`): The  │
│   │              │ answer to the given  │ final answer to the  │
│   │              │ problem.             │ problem              │
│   └──────────────┴──────────────────────┴──────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | gpt-4.1-mini
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Runs web searches for you.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
            ┃ Name          ┃ Description      ┃ Arguments       ┃
            ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
            │ web_search    │ Searches the web │ query           │
            │               │ for your query.  │ (`string`):     │
            │               │                  │ Your query      │
            │ visit_webpage │ Visits a webpage │ url (`string`): │
            │               │ at the given url │ The url of the  │
            │               │ and reads its    │ webpage to      │
            │               │ content as a     │ visit.          │
            │               │ markdown string. │                 │
            │               │ Use this to      │                 │
            │               │ browse webpages. │                 │
            │ final_answer  │ Provides a final │ answer (`any`): │
            │               │ answer to the    │ The final       │
            │               │ given problem.   │ answer to the   │
            │               │                  │ problem         │
            └───────────────┴──────────────────┴─────────────────┘

In [9]:
# added this code, remove plot file if exits.
import os
os.path.exists("saved_map.png") and os.remove("saved_map.png")

False

In [ ]:
manager_agent.run(f"""
{request_museums}

Then make me a spatial map of the world using px.scatter_map, with the biggest museums 
represented as scatter points of size depending on visitor count and color depending 
on the average temperature in July.
Save the map to saved_map.png, then return it!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour",
     color_continuous_scale=px.colors.sequential.Magma_r, size_max=15, zoom=1)
fig.show()
final_answer(fig)

Do not invent any numbers! You must only use numbers sourced from the internet.
""")

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Could you give me a sorted list of the top 3 museums in the    │
│ world in 2024,                                                 │
│ along with their visitor count (in millions) that year, and    │
│ the approximate daily temperature                              │
│ in July at their location ?                                    │
│                                                                │
│ Then make me a spatial map of the world using px.scatter_map,  │
│ with the biggest museums                                       │
│ represented as scatter points of size depending on visitor     │
│ count and color depending                                      │
│ on the average temperature in July.                            │
│ Save the map to saved_map.png, then return it!                 │
│                                                                │
│ Here's an example of how to plot and return a map:             │
│ import plotly.express as px                                    │
│ df = px.data.carshare()                                        │
│ fig = px.scatter_map(df, lat="centroid_lat",                   │
│ lon="centroid_lon", text="name", color="peak_hour",            │
│      color_continuous_scale=px.colors.sequential.Magma_r,      │
│ size_max=15, zoom=1)                                           │
│ fig.show()                                                     │
│ final_answer(fig)                                              │
│                                                                │
│ Do not invent any numbers! You must only use numbers sourced   │
│ from the internet.                                             │
│                                                                │
╰─ OpenAIServerModel - gpt-4.1-mini ─────────────────────────────╯

────────────────────────── Initial plan ──────────────────────────
Here are the facts I know and the plan of action that I will 
follow to solve the task:
```
---
## Facts survey
### 1.1. Facts given in the task
- The task requires the top 3 museums in the world in 2024 ranked 
by visitor count.
- We need the visitor count (in millions) for those museums for 
the year 2024.
- We also need the approximate daily temperature in July at the 
location of each of these museums.
- The task specifies the creation of a spatial map of the world 
using `px.scatter_map`:
  - Scatter points correspond to the museums.
  - Point size depends on visitor count.
  - Point color depends on average July temperature.
- The map should be saved to a file named `saved_map.png`.
- An example snippet for using `px.scatter_map` with the necessary
parameters is provided.
- We are explicitly instructed NOT to invent any numbers, only use
numbers sourced from the internet.

### 1.2. Facts to look up
- The list of the top 3 museums worldwide by visitor count in 
2024:
  - Their exact names.
  - Their official or reliable visitor count numbers for 2024 (in 
millions).
- Geographical coordinates (latitude and longitude) of each 
museum.
- Approximate average daily temperature in July for the location 
(city) of each museum:
  - This data is likely available via climate data websites or 
meteorological services.
- Optionally, check current 2024 visitor numbers through recent 
reports, museum official websites, or authoritative sources like 
The Art Newspaper or other cultural statistics aggregators.
- Locations can be verified via official museum websites, Google 
Maps, or geographical databases.

### 1.3. Facts to derive
- From museum location, derive latitude and longitude coordinates 
required for plotting on the map.
- Using the location's climate data, compute or retrieve the 
average daily temperature for July (if only monthly averages are 
available, use that as an approximation).
- Convert visitor counts to a consistent unit (millions) if 
needed.
- Prepare a dataset combining museum name, visitor count, 
coordinates, and July temperature for plotting with 
`px.scatter_map`.
- Determine appropriate sizing and coloring scales for the scatter
points based on the provided metrics.

## Plan
1. Use the web_agent to search for the current top 3 museums 
worldwide by visitor count in 2024. Collect:
   - Museum names.
   - Visitor count data in millions.
2. For each museum, look up its city/location to obtain 
geographical coordinates (latitude and longitude).
3. Search for climate data for each museum's city to get the 
average daily temperature in July.
4. Consolidate the data into a structured form including museum 
name, visitor count, latitude, longitude, and July temperature.
5. Use `plotly.express.scatter_map` to create a spatial map:
   - Use latitude and longitude for positions.
   - Set scatter point size proportional to visitor count.
   - Set scatter point color mapped to average July temperature.
6. Save the generated map figure to `saved_map.png`.
7. Return the figure using the `final_answer` tool.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will first perform a web search using the web_agent to 
find the top 3 museums in the world in 2024 by visitor count,     
along with their visitor counts. This is the foundational data I  
need before proceeding to look up location and climate.           
                                                                  
Code:                                                             
```py                                                             
task = """Please search for and provide me the current top 3      
museums worldwide in 2024 ranked by visitor count,                
including the official or reliable visitor counts (in millions)   
for the year 2024.                                                
Please include the museum name and the city or country where the  
museum is located."""                                             
answer = web_agent(task=task)                                     
print(answer)                                                     
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  task = """Please search for and provide me the current top 3    
  museums worldwide in 2024 ranked by visitor count,              
  including the official or reliable visitor counts (in           
  millions) for the year 2024.                                    
  Please include the museum name and the city or country where    
  the museum is located."""                                       
  answer = web_agent(task=task)                                   
  print(answer)                                                   
 ────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ─────────────────────────────────────── 
  import re                                                       
                                                                  
  # Look for visitor numbers and museum names in the Art          
  Newspaper content from before, extract relevant text            
  visitor_data_pattern =                                          
  re.compile(r'(\b[A-Za-z\s]+(?:Museum|Gallery)[A-Za-z\s]*?)\s*\  
  (.*?total (\d{1,3},\d{3},\d{3}|\d{1,3},\d{3}|\d{1,3}\.\d{1,2}   
  million|\d+\.\d+ million|\d+ million|\d+,\d+)(.*?)\)')          
  matches = visitor_data_pattern.findall(content1)                
  print("Number of matched museums with visitor data:",           
  len(matches))                                                   
  for m in matches[:5]:  # print first 5 matches for inspection   
      print(m)                                                    
                                                                  
  # Also extract any top museum mention lines from the            
  Wanderlust Magazine content                                     
  lines = content2.split('\n')                                    
  top_museums_lines = [line for line in lines if 'million         
  visitors' in line or 'visitors' in line]                        
  print("Some lines with visitor mentions from Wanderlust         
  Magazine:")                                                     
  for line in top_museums_lines[:5]:                              
      print(line)                                                 
 ────────────────────────────────────────────────────────────────

Execution logs:
Number of matched museums with visitor data: 3
('Shanghai Museum', '1,856,811', '')
('National Gallery of Victoria', '2,840,257', '')
('Smithsonian American Art Museum', '1,521,864', '')
Some lines with visitor mentions from Wanderlust Magazine:
With 8.7 million visitors last year, Paris’ world-famous Louvre 
Museum, home to great works of art like Leonardo da Vinci’s Mona 
Lisa and Venus de Milo, topped the list, according to [The Art 
Newspaper](https://www.theartnewspaper.com/2025/04/01/the-worlds-m
ost-visited-museums-2024-).
In second place, with 6.8 million visitors was the Vatican 
Museums, followed closely by London’s British Museum with 6.4 
million guests. The fourth most popular museum was The 
Metropolitan Museum of Art, famed in both cultural and pop 
cultural circles thanks to the Met Gala and its appearance in 
films and TV shows like *Gossip Girl*, which had 5.7 million 
visitors. Rounding out the top five was another London museum, 
Bankside’s Tate Modern.

Out: None

[Step 3: Duration 5.04 seconds| Input tokens: 13,726 | Output 
tokens: 669]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

> Note, the map will look a bit different as we are only searching for 3 locations.

In [ ]:
fig = manager_agent.python_executor.state["fig"]
fig.update_layout(width=700, height=700)
fig.show()